In [140]:
"""
import pandas as pd
df = pd.read_csv("./qingbao.csv",  on_bad_lines='skip')
res = ""
for i in df['origin']:
    res += i + "\n"
with open("./res.txt", "w", encoding='utf-8') as f:
    f.write(res)
"""
import pandas as pd
from GetRelationShips import GetRelationShips
from GetEventType import GetEventType
from GetEventDetail import GetEventDetail
from load_model import chatglm
from extractJson import extractJson, JsonValidate
import json

/root/anaconda3/envs/ChatGLM2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [141]:
df = pd.read_csv("./qingbao.csv",  on_bad_lines='skip')
res = []
for i in df['origin']:
    res.append(i)

In [142]:
model = chatglm()

Loading checkpoint shards: 100%|██████████| 7/7 [00:13<00:00,  1.86s/it]


In [ ]:
cnt = 0
ana_res = []
output = []
for i in res:
    cnt += 1
    print(f"progress:{cnt}/100")
    # print(f"content:{i}\n\n")
    relation = GetRelationShips(i, model)
    # print(f"relationships={relation}")
    relationJson = extractJson(relation)
    relationJson = relationJson if JsonValidate(relationJson) else 'Json Invalidate'
    # print(f"Json inside={relationJson}")
    if not JsonValidate(relationJson):
        ana_res.append({"事件描述": i, "事件关系": "Json Invalidate"})
        continue
    
    output.append(relationJson)
    with open("./relationJson.json", "w", encoding='utf-8') as f:
        json.dump(output, f)
    
    try:
        pos = 0
        relationJson = json.loads(relationJson)
        # print(f"relationJson={relationJson}")
        for item in relationJson["因果关系"]:
            reason = item.split("->")[0]
            result = item.split("->")[1]
            reason_type = GetEventType(reason, model)
            result_type = GetEventType(result, model)
            relationJson["因果关系"][pos] = {
                "原因": {
                    "事件类型": reason_type,
                    "事件细节": GetEventDetail(reason, reason_type, model),
                    "事件描述": reason
                },
                "结果": {
                    "事件类型": result_type,
                    "事件细节": GetEventDetail(result, result_type, model),
                    "事件描述": result
                }
            }
            pos += 1

        pos = 0
        for item in relationJson["顺承关系"]:
            reason = item.split("->")[0]
            result = item.split("->")[1]
            reason_type = GetEventType(reason, model)
            result_type = GetEventType(result, model)
            relationJson["顺承关系"][pos] = {
                "前一事件": {
                    "事件类型": reason_type,
                    "事件细节": GetEventDetail(reason, reason_type, model),
                    "事件描述": reason
                },
                "后一事件": {
                    "事件类型": result_type,
                    "事件细节": GetEventDetail(result, result_type, model),
                    "事件描述": result
                }
            }
            pos += 1
    except Exception as e:
        # relationJson = str(e)
        print(f"ERROR:{e}")
        continue
    
    ana_res.append({"事件描述": i, "事件分析结果": relationJson})
    if cnt >= 100:
        break
    with open("./analysis_result.json", 'w', encoding='utf-8') as f:
        json.dump(ana_res, f, indent=4)

progress:1/100
progress:2/100
progress:3/100
ERROR:'因果关系'
progress:4/100
ERROR:Expecting ',' delimiter: line 4 column 1 (char 302)
progress:5/100
ERROR:Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
progress:6/100
ERROR:Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
progress:7/100
ERROR:list index out of range
progress:8/100
progress:9/100
ERROR:list index out of range
progress:10/100
ERROR:Expecting ':' delimiter: line 3 column 3 (char 69)
progress:11/100
ERROR:list index out of range
progress:12/100
ERROR:'因果关系'
progress:13/100
progress:14/100
ERROR:Expecting ',' delimiter: line 8 column 42 (char 227)
progress:15/100
progress:16/100
progress:17/100
progress:18/100
progress:19/100


查看事件图谱结果

In [ ]:
import pprint
with open("./analysis_result.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
pprint.pprint(data)

下面是对主题抽取的结果做处理(去除意外词重复词和同义词)的代码

In [120]:
import pprint
import json
with open("./EventType.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

import pandas as pd

dic = {"type": []}
for i in data:
    dic['type'].append(i)
pd.DataFrame(dic).to_csv("./EventType.csv")

# pprint.pprint(data)

In [136]:
from word_similarity import WordSimilarity2010
import tqdm
import json
with open("./type_result.json", 'r', encoding='utf-8') as f:
    data1 = json.load(f)
data = []
for i in data1:
    # if i != 'NONE':
    if 1:
        data.append(i)
ws_tool = WordSimilarity2010()
eps = 0.9
def is_same(a, b):
    sim_b = ws_tool.similarity(a, b)
    return sim_b>eps

import ast

def is_list(string):
    try:
        result = ast.literal_eval(string)
        return isinstance(result, list)
    except (SyntaxError, ValueError):
        return False

unique_data = []
for i in tqdm.tqdm(data):
    dup = False
    for k in i:
        if k in 'abcdefghijklmnopqretuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ':
            dup = True
            print(i)
            break
    for j in unique_data:
        if i in j:
            dup = True
            break
    if not dup:
        unique_data.append(i)

unique_data_outof_number_and_badF = []
for i in unique_data:
    try:
        kk = int(i)
    except:
        if not is_list(str(i)):
            unique_data_outof_number_and_badF.append(i)
        else:    
            # print(i)
            pass

def remove_duplicates(lst):
    result = []
    n = len(lst)

    for i in tqdm.tqdm(range(n)):
        is_duplicate = False
        for j in range(i + 1, n):
            # print(lst[i], lst[j])
            if is_same(lst[i], lst[j]):
                is_duplicate = True
                # print(lst[i], lst[j])
                break
        if not is_duplicate:
            result.append(lst[i])

    return result

def check_If_duplicates(lst):
    n = len(lst)

    for i in tqdm.tqdm(range(n)):
        is_duplicate = False
        for j in range(i + 1, n):
            # print(lst[i], lst[j])
            if is_same(lst[i], lst[j]):
                return True
    return False

unique_data_outof_final = remove_duplicates(unique_data_outof_number_and_badF)
    

100%|██████████| 241/241 [00:00<00:00, 133336.93it/s]


NONE
NONE
NONE


100%|██████████| 224/224 [00:00<00:00, 4449.16it/s]


In [137]:
len(data), len(unique_data), len(unique_data_outof_number_and_badF), len(unique_data_outof_final), check_If_duplicates(unique_data_outof_final)

100%|██████████| 224/224 [00:00<00:00, 4306.21it/s]


(241, 224, 224, 224, False)

In [138]:
pd.Series(unique_data_outof_final).to_csv("./Event_Type_final.csv")

In [139]:
pd.Series(unique_data_outof_final).head(20)

0       窃取
1       渗透
2       打击
3     利用木马
4       销毁
5       攻击
6       复制
7       发现
8       感染
9       建立
10      建设
11      混淆
12      学习
13      自查
14      分析
15      状态
16      装载
17      信息
18     重定向
19      制造
dtype: object

查看type_result.json文件

In [ ]:
import pprint
import json
with open("./type_result.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
pprint.pprint(data)